In [1]:
import quant_inf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from tqdm import tqdm

dict_color = {
    'quantum':'#1f77b4',
    'quant_diag':'#ff7f0e',
    'quant_greedy': '#2ca02c',
    'logdet':'#d62728',
    'TRW':'#9467bd',
    'quant_exp':'#8c564b',
    'C1': '#e377c2',
    'C2': '#7f7f7f',
    'C3': '#bcbd22',
    'C4': '#17becf'}


In [2]:
sns.set_context("notebook", font_scale=1)

# Experiment 1

In [3]:
def get_result_experiment_1(
        d:int,
        n_sample:int):
    """For k between d+1 and 2^d, computes quantum and quant_greedy
    relaxation of logp with k features.

    Returns:
        pandas.core.frame.DataFrame: a dataframe with columns
        ['k','relaxation','logp_bound','true_logp','l1_error','d','graph']
    
    Args:
        d (int): number of variables
        max_features_greedy (int): number of features selected for greedy algorithm
        n_sample (int): number of sample for each size of feature vector
    """
    
    results = pd.DataFrame(columns=['k','relaxation','logp_bound','true_logp','l1_error','d','graph'])

    indep_features = [{i} for i in range(1,d+1)]

    tree_features = (
        [{i} for i in range(1,d+1)] 
        + [{i,i+1} for i in range(1,d)]
    )
    complete_features = (
        [{i} for i in range(1,d+1)]
        + [{i,j} for i in range(1,d+1) for j in range(i+1,d+1)]
    )

    dict_features = {
        "Independent variables":indep_features,
        "Tree":tree_features,
        "Complete graph":complete_features
    }

    all_features = (
        [set()] 
        + [{i} for i in range(1,d+1)]
        + [{i,j} for i in range(1,d+1) for j in range(i+1,d+1)]
        + [{i,j,k} for i in range(1,d+1) for j in range(i+1,d+1) for k in range(j+1,d+1)]
        + [{i,j,k,l} for i in range(1,d+1) for j in range(i+1,d+1) for k in range(j+1,d+1) for l in range(k+1,d+1)]
        + [{i,j,k,l,m} for i in range(1,d+1) for j in range(i+1,d+1) for k in range(j+1,d+1) for l in range(k+1,d+1) for m in range(l+1,d+1)]
    )
    
    n = len(all_features)

    for graph in ["Independent variables", "Tree", "Complete graph"]:             
        for _ in tqdm(range(n_sample)):

            coefficients = quant_inf.tools.random_coefficients_gaussian(
                graph_features=dict_features[graph]
                )
            
            quant_greedy_inference = quant_inf.algorithms.QuantGreedyRelaxation(
                coefficients=coefficients,
                d=d)
            
            exact_inference = quant_inf.algorithms.ExactBruteForce(
                    coefficients=coefficients,
                    d=d,
                    features=all_features[:d+1])
            exact_inference.solve()
            
            for k in range(d+1,n+1):
                quant_inference = quant_inf.algorithms.QuantumRelaxation(
                    coefficients=coefficients,
                    d=d,
                    features=all_features[:k])
                quant_inference.solve(tol=1e-4)

                results.loc[len(results)] = [
                    k,
                    "quantum",
                    quant_inference.log_partition,
                    exact_inference.log_partition,
                    quant_inference.l1_error(exact_inference.marginals),
                    d,
                    graph
                    ]
                
                if k == d+1:
                    quant_greedy_inference.solve(number_extra_features=0,tol=1e-4)
                else:
                    quant_greedy_inference._select_feature(tol=1e-4)
                results.loc[len(results)] = [
                    k,
                    "quant_greedy",
                    quant_greedy_inference.log_partition,
                    exact_inference.log_partition,
                    quant_greedy_inference.l1_error(exact_inference.marginals),
                    d,
                    graph
                    ]
                
    return results

In [4]:
def plot_experiment_1(result_experiment,graph:str,title:str="",file:str=""):
    fig = plt.figure(figsize=(5,4.5))
    
    to_plot = (
        result_experiment
        .query('graph == @graph')
        .assign(normalized_error_in_bound=lambda x: (x.logp_bound - x.true_logp)/x.d)
    )
    ax1 = plt.subplot(3,1,(1,2))
    sns.lineplot(
        data=to_plot,
        x="k",
        y="normalized_error_in_bound",
        hue="relaxation",
        palette=dict_color,
        errorbar=('sd',1),
        ax=ax1
    )
    ax1.set_ylabel("Normalized error in bound")
    ax1.set_xlabel(None)
    plt.setp(ax1.get_xticklabels(), visible=False)
    ax1.axhline(y=0,color="grey")
    
    ax2 = plt.subplot(3,1,3,sharex=ax1)
    sns.lineplot(
        data=to_plot,
        x="k",
        y="l1_error",
        hue="relaxation",
        palette=dict_color,
        errorbar=('sd',1),
        ax=ax2,
        legend=False
    )
    ax2.set_ylabel("Error in\n marginals")
    ax2.set_xlabel("Number of features")
    
    fig.suptitle(title)
    fig.tight_layout()
    if file != "":
        plt.savefig(file)
    plt.show()

In [ ]:
d=5
result_experiment_1= get_result_experiment_1(
    d=d,
    n_sample=10
    )

## Plots

In [ ]:
print("Independent variables")
plot_experiment_1(result_experiment=result_experiment_1,graph="Independent variables")
print("Tree")
plot_experiment_1(result_experiment=result_experiment_1,graph="Tree")
print("Complete graph")
plot_experiment_1(result_experiment=result_experiment_1,graph="Complete graph")